In [257]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import argparse
from utils import cfh, logger, Data, bool_ext, checkCorrelations, generate_masks_from_ppi
import os
import argparse
import random
from utils import cfh, logger, Data, bool_ext, checkCorrelations, generate_masks_from_ppi
from biomodels import BioCitrus
import torch
import numpy as np
import sys
from biomodels import weightConstraint
from utils import logger, get_minibatch, evaluate, EarlyStopping, shuffle_data
from tqdm import tqdm
from pathlib import Path
from IPython.display import clear_output
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore") ##This is bad but temporary

In [3]:
parser = argparse.ArgumentParser()

In [4]:
parser.add_argument(
    "--input_dir", 
    help="directory of input files", 
    type=str, 
    default="./data"
)
parser.add_argument(
    "--output_dir",
    help="directory of output files",
    type=str,
    default="./output",
)

parser.add_argument(
    "--algo", 
    help="clustering algorithm to use on the portein-protein network (DPCLUS, MCODE, COACH)", 
    type=str, 
    default='COACH'
)


parser.add_argument(
    "--learning_rate", 
    help="learning rate for Adam", 
    type=float, 
    default=1e-2
)
parser.add_argument(
    "--max_iter", 
    help="maximum number of training iterations", 
    type=int, 
    default=300
)
parser.add_argument(
    "--max_fscore",
    help="Max F1 score to early stop model from training",
    type=float,
    default=0.7
)
parser.add_argument(
    "--batch_size", 
    help="training batch size", 
    type=int, 
    default=100
)
parser.add_argument(
    "--test_batch_size", 
    help="test batch size", 
    type=int, 
    default=100
)
parser.add_argument(
    "--test_inc_size",
    help="increment interval size between log outputs",
    type=int,
    default=256
)
parser.add_argument(
    "--dropout_rate", 
    help="dropout rate", 
    type=float, 
    default=0.1
)

parser.add_argument(
    "--weight_decay", 
    help="coefficient of l2 regularizer", 
    type=float, 
    default=1e-2
)
parser.add_argument(
    "--activation",
    help="activation function used in hidden layer",
    type=str,
    default="tanh",
)
parser.add_argument(
    "--patience", 
    help="earlystopping patience", 
    type=int, 
    default=20
)
parser.add_argument(
    "--mask01",
    help="wether to ignore the float value and convert mask to 01",
    type=bool_ext,
    default=True,
)
parser.add_argument(
    "--gep_normalization", 
    help="how to normalize gep", 
    type=str, 
    default="scaleRow"
)

parser.add_argument(
    "--cancer_type",
    help="whether to use cancer type or not",
    type=bool_ext,
    default=False,
)
parser.add_argument(
    "--train_model",
    help="whether to train model or load model",
    type=bool_ext,
    default=True,
)
parser.add_argument(
    "--dataset_name",
    help="the dataset name loaded and saved",
    type=str,
    default="dataset_CITRUS",
)
parser.add_argument(
    "--tag", 
    help="a tag passed from command line", 
    type=str, 
    default=""
)
parser.add_argument(
    "--run_count", 
    help="the count for training", 
    type=str, 
    default="1"
)

parser.add_argument(
    "--ppi_weights", 
    help="", 
    type=bool_ext, 
    default=False
)

parser.add_argument(
    "--verbose", 
    help="", 
    type=bool_ext, 
    default=False
)

parser.add_argument(
    "--constrain", 
    help="force weight and biases to be strictly non-negative", 
    type=bool_ext, 
    default=False
)

parser.add_argument(
    "--biases", 
    help="enable all nn.Linear biases", 
    type=bool_ext, 
    default=True
)

parser.add_argument(
    "--sparse", 
    help="only use SIGNOR data, resulting in sparser connections", 
    type=bool_ext, 
    default=False
)

args = parser.parse_args([])

data = Data(
    fGEP_SGA = 'data/CITRUS_GEP_SGAseparated.csv',
    fgene_tf_SGA = 'data/CITRUS_gene_tf_SGAseparated.csv',
    fcancerType_SGA = 'data/CITRUS_canType_SGAseparated.csv',
    fSGA_SGA = 'data/CITRUS_SGA_SGAseparated.csv',
    cancer_type='BRCA'
)

train_set, test_set = data.get_train_test()
args.gep_size = train_set['gep'].shape[1]
args.tf_gene = data.gene_tf_sga.values.T
args.can_size = len(np.unique(data.cancer_types))


sga_mask, sga_weights, tf_mask, tf_weights = generate_masks_from_ppi(sga = data.sga_sga, tf = data.gene_tf_sga, clust_algo=args.algo, sparse=args.sparse)


sga_mask = sga_mask
sga_weights = sga_weights.t()
tf_mask = tf_mask.t()
tf_weights = tf_weights


(utils.py : 371) -    DEBUG | Loaded 352251 edges from the SIGNOR and SNAP Networks
(utils.py : 395) -    DEBUG | Using induced overlap network with 10270 common genes
(utils.py : 489) -     INFO | Using dense COACH clustering algorithm
(utils.py : 550) -     INFO | Generated sga-ppi mask with 3862 clusters and 6533 edges
(utils.py : 569) -     INFO | Generated ppi-tf mask with 3862 clusters and 268 edges


In [5]:
from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
from captum.attr import IntegratedGradients

In [6]:
models = []
for i in range(30):
    
    model = BioCitrus(
        args = args, 
        sga_ppi_mask = sga_mask, 
        ppi_tf_mask = tf_mask, 
        sga_ppi_weights = None, 
        ppi_tf_weights = None,
        enable_bias = args.biases
    )

    model.load_state_dict(torch.load(f'/ix/hosmanbeyoglu/kor11/CITRUS_models/BRCA_{i}.pth', 
                                map_location=torch.device('cpu')))
    
    model.eval()
    
    models.append(model)
    clear_output(wait=True)

(biomodels.py : 106) -    DEBUG | sga_layer.0.mask | False | (3862, 11998)
(biomodels.py : 106) -    DEBUG | sga_layer.0.weight | True | (3862, 11998)
(biomodels.py : 106) -    DEBUG | sga_layer.0.bias | True | (3862,)
(biomodels.py : 106) -    DEBUG | tf_layer.0.mask | False | (320, 3862)
(biomodels.py : 106) -    DEBUG | tf_layer.0.weight | True | (320, 3862)
(biomodels.py : 106) -    DEBUG | tf_layer.0.bias | True | (320,)
(biomodels.py : 106) -    DEBUG | gep_output_layer.weight | True | (5541, 320)
(biomodels.py : 106) -    DEBUG | gep_output_layer.bias | True | (5541,)
(biomodels.py : 108) -    DEBUG | Constraints Enabled: False
(biomodels.py : 109) -    DEBUG | Biases Enabled: True


In [7]:
X = torch.tensor(test_set['sga'])
Y = test_set['gep']

In [8]:
all_attr_scores = np.load('all_attr_scores.npy')

In [9]:
models = np.array(models)[[21, 20, 10, 17, 5, 11, 12, 14, 6, 25]]

In [10]:
dff = pd.DataFrame(all_attr_scores[0], 
    columns=data.sga_sga.columns, index=data.gep_sga.columns).sum(0).sort_values(ascending=False)

In [11]:
df = pd.DataFrame([data.sga_sga.sum().loc[dff.index].values, dff.values], 
                  columns=dff.index, index=['alt_freq', 'int_grad']).T

In [12]:
df['int_grad'] = ((df['int_grad'] - df['int_grad'].min())/(df['int_grad'].max()-df['int_grad'].min()))*10000

In [13]:
df['int_grad'].mean()

1.6811278043547329

In [14]:
df['int_grad'] = np.log10(df['int_grad'])
df['alt_freq'] = np.log10(df['alt_freq'])

In [15]:
# plt.figure(figsize=(10, 8))
# ax = sns.scatterplot(data=df, x='alt_freq', y='int_grad')
# plt.xlabel('Log10 Alteration Frequency')
# plt.ylabel('Log10 Integrated Gradient Importance')
# # plt.savefig('frequency_plots.png', dpi=180)
# plt.show()

In [16]:
def tf_activity(model, target_gene):
    lc = LayerConductance(model, model.gep_output_layer)
    ix = list(data.gep_sga.columns).index(target_gene)
    a = lc.attribute(X, n_steps=5, attribute_to_layer_input=False, target=[ix]*len(X))
    
    ig_attr_test_sum = a.detach().numpy().sum(0)
    ig_attr_test_norm_sum = ig_attr_test_sum / np.linalg.norm(ig_attr_test_sum, ord=1)
    
    g = np.array(data.gene_tf_sga.columns)[np.where(ig_attr_test_norm_sum != 0)[0]]
    at = ig_attr_test_norm_sum[np.where(ig_attr_test_norm_sum != 0)[0]]
    
    df = pd.DataFrame([g, at]).T
    df.columns = ['TF', 'score']
    
    return df.sort_values(by='score', ascending=False)

In [17]:
with open('/ihome/hosmanbeyoglu/kor11/tools/CITRUS/COACH_clusters_large.txt', "r") as fh:
        lines = fh.readlines()
        clusterindex_to_genes = {}
        for i, c in enumerate(lines):
            clustlist = c.strip().split(" ")
            if len(c) == 0:
                continue
            clusterindex_to_genes[i] = clustlist 

from collections import defaultdict
gene_to_clusterindices = defaultdict(list) ## 'MAPK1':[0, 75, 129, 373]

## Create mapping between genes and the protein clusters
for c in clusterindex_to_genes.keys():
    for g in clusterindex_to_genes[c]:
        gene_to_clusterindices[g].append(c)  

In [18]:
cancers = ['BLCA', 'BRCA', 'CESC', 'COAD', 'ESCA', 'GBM', 'HNSC', 'KIRC',
       'KIRP', 'LIHC', 'LUAD', 'LUSC', 'PCPG', 'PRAD', 'STAD', 'THCA',
       'UCEC']

In [19]:
# r = []
# for cancer in cancers:
#     m = np.load(f'./metrics/{cancer}_shuffled_metrics.npy')
#     mean_loss, mean_corr = np.around(np.array([x[-1] for x in m]).mean(0), 3)
#     std_loss, std_corr = np.round(np.array([x[-1] for x in m]).std(0), 3)
#     r.append((cancer, f'{mean_corr}+/-{std_corr}', f'{mean_loss}+/-{std_loss}'))
# pd.DataFrame(r, columns = ['cancer type', 'mse', 'pearson'])

In [20]:
data = Data(
    fGEP_SGA = 'data/CITRUS_GEP_SGAseparated.csv',
    fgene_tf_SGA = 'data/CITRUS_gene_tf_SGAseparated.csv',
    fcancerType_SGA = 'data/CITRUS_canType_SGAseparated.csv',
    fSGA_SGA = 'data/CITRUS_SGA_SGAseparated.csv',
    cancer_type='BRCA'
)

train_set, test_set = data.get_train_test()


In [21]:
df = data.sga_sga
X = df.values
X.shape

(720, 11998)

In [22]:
X1 = df[(df['SM_PIK3CA']==1)]
X1.shape

(265, 11998)

In [23]:
X0 = df[(df['SM_PIK3CA']==0)]
X0.shape

(455, 11998)

In [24]:
tf_profiles1 = np.load('tf_profiles_1.npy')
tf_profiles2 = np.load('tf_profiles_2.npy')


In [25]:
r = [tf_profiles1, tf_profiles2]

In [26]:
from scipy.stats import mannwhitneyu

In [27]:
rf = pd.DataFrame([data.gene_tf_sga.columns, 
        [mannwhitneyu(r[0][:, i][:], r[1][:, i]).pvalue * 5541 for i in range(320)]]).T.sort_values(by=1)

rf.columns = ['TF', 'P']
rf['EFFECTSIZE'] = abs(tf_profiles1.mean(0)  - tf_profiles2.mean(0))
print(rf[rf.P < 0.05].shape)
# rf[rf.AdjPvalue < 0.05].sort_values(by='foldchange', ascending=False)
rf = rf[rf.P < 1]

(50, 3)


In [28]:
rf.P = rf.P.astype(np.float)

In [29]:
rf

,TF,P,EFFECTSIZE
294,HEY1,1.247146e-108,1.288033e-01
128,TCF12,5.787345e-35,7.084708e-01
121,NFYC,5.657469e-08,1.046785e-07
153,FLI1,1.543887e-06,5.049044e-01
83,ERG,1.681252e-06,0.000000e+00
...,...,...,...
257,ETV1,6.305362e-01,0.000000e+00
47,RUNX1,8.064497e-01,1.074662e-06
154,HIC1,8.492413e-01,4.700601e-03
26,HIVEP1,8.729153e-01,1.254514e-06


In [92]:
import requests
import json 
import pandas as pd

class Enrichr(object):
    
    def __init__(self):
        self.ENRICHR_URL_ADDLIST = 'https://maayanlab.cloud/Enrichr/addList'
        self.ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/enrich'
        self.QUERY_STR = '?userListId=%s&backgroundType=%s'
        self.libraries = [
            'VirusMINT', 
            'GO_Biological_Process_2021', 
            'MSigDB_Hallmark_2020', 
            'KEGG_2021_Human', 
            'Reactome_2016']
        
    
    def _addlist(self, geneset):
        genes_str = '\n'.join(geneset)
        description = ''
        payload = {
            'list': (None, genes_str),
            'description': (None, description)
        }
        response = requests.post(self.ENRICHR_URL_ADDLIST, files=payload)
        data = json.loads(response.text)
        
        return data['userListId']
    
    def get_enrichment_results(self, geneset, gene_set_library = 'GO_Biological_Process_2021'):
        user_list_id = self._addlist(geneset)
        response = requests.get(
        self.ENRICHR_URL + self.QUERY_STR % (user_list_id, gene_set_library))
        data = json.loads(response.text)
        df = pd.DataFrame(data[gene_set_library])[[1, 2, 3, 4, 5, 6]]
        df.columns = ['Terms', 'Pval', 'OddsRatio', 'Score', 'Genes', 'AdjPval']
        
        return df[round(df.AdjPval, 3) < 0.05].sort_values(by='AdjPval')

In [31]:
model = models[0]
target_gene = 'PIK3CA'

In [108]:
list(data.sga_sga.columns).index('SM_PIK3CA'), list(data.sga_sga.columns).index('SCNA_PIK3CA')

(211, 7650)

In [110]:
np.where(model.sga_layer[0].mask[:, 211] == 1)[0].__len__()

19

In [248]:
np.where(model.sga_layer[0].mask[:, 7650] == 1)[0].__len__()

19

In [93]:
enrichr = Enrichr()

In [220]:
[' | '.join(enrichr.get_enrichment_results(geneset=set(clusterindex_to_genes[i]), gene_set_library='KEGG_2021_Human').Terms[:2].values) 
        for i in np.where(model.sga_layer[0].mask[:, 211] == 1)[0]]

['T cell receptor signaling pathway | Natural killer cell mediated cytotoxicity',
 'Inflammatory mediator regulation of TRP channels | Insulin resistance',
 'ErbB signaling pathway | Chronic myeloid leukemia',
 'JAK-STAT signaling pathway | Pathways in cancer',
 'Pathways in cancer | Proteoglycans in cancer',
 'Focal adhesion | Neurotrophin signaling pathway',
 'ErbB signaling pathway | Renal cell carcinoma',
 'ErbB signaling pathway | Chronic myeloid leukemia',
 'Hepatitis C | Proteoglycans in cancer',
 'Thyroid hormone signaling pathway | Pathways in cancer',
 'Autophagy | Neurotrophin signaling pathway',
 'Pathways in cancer | Transcriptional misregulation in cancer',
 'Growth hormone synthesis, secretion and action | Insulin signaling pathway',
 'PI3K-Akt signaling pathway | MicroRNAs in cancer',
 'TNF signaling pathway | NF-kappa B signaling pathway',
 'Prostate cancer | PI3K-Akt signaling pathway',
 'Pathways in cancer | Thyroid hormone signaling pathway',
 'Insulin resistance | 

In [229]:
libraries = [
    'VirusMINT', 
    'GO_Biological_Process_2021', 
    'MSigDB_Hallmark_2020', 
    'KEGG_2021_Human', 
    'Reactome_2016']

In [247]:
model.sga_layer[0].mask.shape

torch.Size([3862, 11998])

In [113]:
np.where(model.sga_layer[0].mask[:, 211] == 1)[0]

array([  32,  304,  413,  437, 1379, 1388, 1725, 1784, 1841, 1997, 2100,
       2118, 2248, 2693, 2888, 2967, 3293, 3628, 3852])

In [112]:
model.tf_layer[0].mask.shape

torch.Size([320, 3862])

In [123]:
cluster2tf = dict(zip(np.where(model.sga_layer[0].mask[:, 211] == 1)[0], [np.where(model.tf_layer[0].mask[:, i] == 1)[0].tolist() for i in np.where(model.sga_layer[0].mask[:, 211] == 1)[0]]))

In [134]:
tfs = set()
for k, v in cluster2tf.items():
    for i in v:
        tfs.add(i)

In [136]:
tfs.__len__()

44

In [249]:
data.gene_tf_sga.columns.__len__()

320

In [141]:
data.gene_tf_sga.columns[list(tfs)]

Index(['SMARCC1', 'PPARG', 'FOXO3', 'TCF12', 'FOXH1', 'BRCA1', 'MAFK', 'RUNX2',
       'TP73', 'YY1', 'FLI1', 'FOXP3', 'CREB1', 'TP53', 'HIF1A', 'HEY1',
       'MECOM', 'SMARCC2', 'SMAD3', 'RUNX1', 'JUN', 'E2F5', 'AR', 'MYC',
       'TGIF1', 'STAT3', 'ESR2', 'SMAD4', 'STAT1', 'CEBPA', 'CEBPB', 'GLI3',
       'PLAGL1', 'SMAD1', 'ZEB1', 'ATF3', 'RELA', 'RUNX3', 'ESR1', 'NFYC',
       'CEBPD', 'NR3C1', 'SMAD2', 'E2F4'],
      dtype='object')

In [232]:
X1 = df[(df['SM_PIK3CA']==1) | (df['SCNA_PIK3CA']==1)]
X0 = df[(df['SM_PIK3CA']==0) & (df['SCNA_PIK3CA']==0)]

X1.shape, X0.shape

((285, 11998), (435, 11998))

In [233]:
attrs_a = []
attrs_b = []
model = models[1]

for xx in [X1, X0]:
    patient = torch.from_numpy(xx.values)
    lc = LayerConductance(model, model.tf_layer)
    ix = list(data.gep_sga.columns).index('PIK3CA')
    a = lc.attribute(patient, n_steps=50, attribute_to_layer_input=True, target=ix)
    b = lc.attribute(patient, n_steps=50, attribute_to_layer_input=False, target=ix)


    ig_attr_test_sum = a.detach().numpy().mean(0)
    ig_attr_test_norm_sum = ig_attr_test_sum / np.linalg.norm(ig_attr_test_sum, ord=1)

    ig_attr_test_sum_b = b.detach().numpy().mean(0)
    ig_attr_test_norm_sum_b = ig_attr_test_sum_b / np.linalg.norm(ig_attr_test_sum_b, ord=1)

    ig_attr_test_sum.shape, ig_attr_test_sum_b.shape
    
    attrs_a.append(ig_attr_test_norm_sum)
    attrs_b.append(ig_attr_test_norm_sum_b)
    

In [239]:
f = pd.DataFrame(attrs_a).T.sort_values(by=0, ascending=False)[:20]
f.columns = ['X1', 'X0']
f['delta'] = abs(f.X1 - f.X0)
f = f.sort_values(by='delta', ascending=False)
f


,X1,X0,delta
1997,0.081387,0.014886,0.066501
3229,0.120188,0.133060,0.012872
3151,0.082730,0.092031,0.009301
3018,0.102026,0.107705,0.005679
1571,0.052090,0.057322,0.005231
3162,0.030771,0.035194,0.004423
2460,0.025059,0.029040,0.003981
53,0.048185,0.050850,0.002665
3767,0.036190,0.038307,0.002117
3600,0.027042,0.028902,0.001860


In [262]:
gene_counter = defaultdict(int)
for i in f.index:
    for g in clusterindex_to_genes[i]:
        gene_counter[g]+=1

In [273]:
pd.DataFrame(gene_counter.items()).set_index(0).sort_values(by=1, ascending=False)[:10] / len(f)

,1
0,
SMARCA4,0.95
HDAC1,0.90
ESR1,0.90
TP53,0.90
MYC,0.85
CREBBP,0.85
EP300,0.80
BRCA1,0.80
JUN,0.80


In [250]:
set(clusterindex_to_genes[1997]).intersection(clusterindex_to_genes[1919])

{'AKT1',
 'BRCA1',
 'CDK8',
 'CREBBP',
 'CTNNB1',
 'EP300',
 'ESR1',
 'FHL2',
 'FOXO3',
 'HDAC1',
 'JUN',
 'MAPK1',
 'MAPK3',
 'MYC',
 'NCOA3',
 'NCOR1',
 'PPARG',
 'SMAD2',
 'SMAD3',
 'SMAD4',
 'SMARCA4',
 'TP53'}

In [243]:
[' | '.join(enrichr.get_enrichment_results(geneset=set(clusterindex_to_genes[i]), gene_set_library='GO_Biological_Process_2021').Terms[:2].values) 
        for i in f.index]

['positive regulation of nucleic acid-templated transcription (GO:1903508) | positive regulation of transcription, DNA-templated (GO:0045893)',
 'transcription initiation from RNA polymerase II promoter (GO:0006367) | DNA-templated transcription, initiation (GO:0006352)',
 'negative regulation of transcription by RNA polymerase II (GO:0000122) | negative regulation of transcription, DNA-templated (GO:0045892)',
 'negative regulation of transcription, DNA-templated (GO:0045892) | regulation of transcription by RNA polymerase II (GO:0006357)',
 'negative regulation of transcription, DNA-templated (GO:0045892) | negative regulation of transcription by RNA polymerase II (GO:0000122)',
 'negative regulation of transcription, DNA-templated (GO:0045892) | negative regulation of transcription by RNA polymerase II (GO:0000122)',
 'positive regulation of transcription, DNA-templated (GO:0045893) | positive regulation of nucleic acid-templated transcription (GO:1903508)',
 'negative regulation of

In [241]:
round(pd.DataFrame(attrs_b, columns=data.gene_tf_sga.columns).T
        # .loc[data.gene_tf_sga.columns[list(tfs)]]
        .sort_values(by=0, ascending=False), 5)[:20]


,0,1
FOXO3,0.25577,0.23517
SMARCC1,0.20519,0.21889
SMARCC2,0.20119,0.18387
E2F1,0.17454,0.18513
DNMT1,0.07626,0.08275
CTCFL,0.06303,0.06964
FOXO1,0.02106,0.02167
KLF6,0.00265,0.00266
TEAD3,0.00017,0.00013
FOXP2,0.00005,0.00004
